Functions available from **DataLakeHelperFunctions**

|Function|Purpose|
|--|--|
|mount_lake_container|Takes a container name and mounts it to Databricks. Prints out the name of the mount point. Uses Service Principle Auth. |
|convert_full_file_path_to_mount_point|Takes a parameter that is a full file path to a file in a Azure Data Lake. Converts the string to use the Databricks mount point to the applicbale container instead.|
|get_latest_modified_file_from_directory|For a given directory, return the file that was last modified|
| convert_container_and_directory_to_mountpoint|Takes the given data lake container name and directory inside the container and returns a path inside the mount point using standard formatting.|
|get_json_df_tangent|Converts a column in a PySpark DataFrame from string to Struct|
|execute_autoflatten|Flattens a JSON column in a PySpark DataFrame|
|execute_autoflatten_json|Receives a Data Frame with a column that contains JSON code as strings, converts it to typed JSON and flattens it. |

Classes available from **DataLakeHelperFunctions**
|Class|Purpose|
|--|--|
|Log4jWrapper|A class that creates a callable instance of the PySpark built in Log4j JVM logging object. Log4j is a Java-based logging utility part of the Apache Logging Services.|

In [ ]:
def mount_lake_container(pAdlsContainerName, pSecretScopeName='KeyVault',  pStorageAccountName='DataLakeStorageAccountName', pSecretClientID='DataLakeAuthServicePrincipleClientID', pSecretClientSecret='DataLakeAuthServicePrincipleClientSecret', pSecretTenantID='DataLakeAuthServicePrincipleTenantID'):
    """
    mount_lake_container:
        Takes a container name and mounts it to Databricks for easy access. 
        Prints out the name of the mount point. 
        Uses a service princple to authenticate.
      
    param pAdlsContainerName: The name of the container that will be mounted.
    param pSecretScopeName: Databricks Secret Scope name.
    param pStorageAccountName: The storage accounts name.
    param pSecretClientID: Name of the key vault secret containing the Service Principle name.
    param pSecretClientSecret: Name of the key vault secret containing the Service Principle Password.
    param pSecretTenantID: Name of the key vault secret containing the Tenant ID
    """
    # Define the variables used for creating connection strings - Data Lake Related
    vAdlsAccountName = dbutils.secrets.get(scope=pSecretScopeName, key=pStorageAccountName) # e.g. "dummydatalake" - the storage account name itself
    vAdlsContainerName = pAdlsContainerName # e.g. rawdata, bronze, silver, gold, platinum etc.
    vMountPoint = "/mnt/datalake_" + vAdlsContainerName #fixed since we already parameterised the container name. Ensures there is a standard in mount point naming

    # Get the actual secrets from key vault for the service principle
    vApplicationId = dbutils.secrets.get(scope=pSecretScopeName, key=pSecretClientID) # Application (Client) ID
    vAuthenticationKey = dbutils.secrets.get(scope=pSecretScopeName, key=pSecretClientSecret) # Application (Client) Secret Key
    vTenantId = dbutils.secrets.get(scope=pSecretScopeName, key=pSecretTenantID) # Directory (Tenant) ID

    # Using the secrets above, generate the URL to the storage account and the authentication endpoint for OAuth
    vEndpoint = "https://login.microsoftonline.com/" + vTenantId + "/oauth2/token" #Fixed URL for the endpoint
    vSource = "abfss://" + vAdlsContainerName + "@" + vAdlsAccountName + ".dfs.core.windows.net/"

    # Connecting using Service Principal secrets and OAuth
    vConfigs = {"fs.azure.account.auth.type": "OAuth", #standard
               "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider", #standard
               "fs.azure.account.oauth2.client.id": vApplicationId,
               "fs.azure.account.oauth2.client.secret": vAuthenticationKey,
               "fs.azure.account.oauth2.client.endpoint": vEndpoint}

    # Mount Data Lake Storage to Databricks File System only if the container is not already mounted
    # First generate a list of all mount points available already via dbutils.fs.mounts()
    # Then it checks the list for the new mount point we are trying to generate.
    if not any(mount.mountPoint == vMountPoint for mount in dbutils.fs.mounts()): 
      dbutils.fs.mount(
        source = vSource,
        mount_point = vMountPoint,
        extra_configs = vConfigs)

    # print the mount point used for troubleshooting in the consuming notebook
    print("Mount Point: " + vMountPoint)

In [ ]:
def convert_full_file_path_to_mount_point(pFullFilePath):
    
    """
    convert_full_file_path_to_mount_point:
        Takes a parameter that is a full file path to a file in a Azure Data Lake. 
        Converts the string to use the Databricks mount point to the applicbale container instead.
        Format of URL expected: https://dianrandddatalake.blob.core.windows.net/rawdata/DummyAutomatedDirectory/2022/02/28/16/30/wwi-dimstockitem.csv
    """
    
    # KeyVault Secret Scope Name - use a variable because it is referenced multiple times
    vSecretScopeName = "KeyVault" # Fixed standardised name. To ensure deployment from DEV to PROD is seemless. 
    
    # Get the storage account name from key vault for use later when altering the full file path received to reference the data lake container mount point instead
    vStorageAccountName = dbutils.secrets.get(scope=vSecretScopeName,key="DataLakeStorageAccountName")
    
    # String to replace in full file path to have it use the mount point instead
    vStorageAccountURL = "https://" + vStorageAccountName + ".blob.core.windows.net/"
    
    # Value to replace the storage account url with
    # Note, this does not contain the suffix with the container name, we get that from the full file path already.
    vStorageAccount_MountPointPrefix = "/mnt/datalake_"

    # Remove the URL of the storage account and replace with the mount point string
    # This variable "pFileFullPath" can now be used to read the applicable file into memory into a dataframe
    pMountPointPath = pFullFilePath.replace(vStorageAccountURL, vStorageAccount_MountPointPrefix)

    return pMountPointPath

In [ ]:
import os
from datetime import datetime

def get_dir_content(pPath):
    """
    get_dir_content:
        For a folder in the data lake, get the list of files it contains, including all subfolders. 
        Return Full File Name as well as Last Modified Date time as a generator object. 
        Output requires conversion into list for consumption.
    """
    #This for loop will check all directories and files inside the provided path
    #For each file it contains, return a 2-D array witht he file name and the last modified date time
    #The consuming code will need to convert the generater object this returns to a list to consume it
    #THe yield function is used to ensure the entire directory contents is scanned. If you used return it would stop after the first object encountered. 
    for dir_path in dbutils.fs.ls(pPath):
        if dir_path.isFile():
            #os.stat gets statistics on a path. st_mtime gets the most recent content modification date time
            yield [dir_path.path, datetime.fromtimestamp(os.stat('/' + dir_path.path.replace(':','')).st_mtime)]
        elif dir_path.isDir() and pPath != dir_path.path:
            #if the path is a directory, call the function on it again to check its contents
            yield from get_dir_content(dir_path.path)

def get_latest_modified_file_from_directory(pDirectory):
    """
    get_latest_modified_file_from_directory:
        For a given path to a directory in the data lake, return the file that was last modified. 
        Uses the get_dir_content function as well.
        Input path format expectation: '/mnt/datalake_rawdata'
            You can add sub directories as well, as long as you use a registered mount point
        Performance: With 588 files, it returns in less than 10 seconds on the lowest cluster size. 
    """
    #Call get_dir_content to get a list of all files in this directory and the last modified date tiem of each
    vDirectoryContentsList =list(get_dir_content(pDirectory))

    #Convert the list returned from get_dir_content into a dataframe so we can manipulate the data easily. Provide it with column headings. 
    df = spark.createDataFrame(vDirectoryContentsList,['FullFilePath', 'LastModifiedDateTime'])

    #Get the latest modified date time scalar value
    maxLatestModifiedDateTime = df.agg({"LastModifiedDateTime": "max"}).collect()[0][0]

    #Filter the data frame to the record with the latest modified date time value retrieved
    df_filtered = df.filter(df.LastModifiedDateTime == maxLatestModifiedDateTime)
    
    #return the file name that was last modifed in the given directory
    return df_filtered.first()['FullFilePath']


In [ ]:
def convert_container_and_directory_to_mountpoint(pContainer, pDirectory):
    """
    convert_container_and_directory_to_mountpoint:
        Takes the given data lake container name and directory inside the container and returns a path inside the mount point using standard formatting. 
    """
    return "/mnt/datalake_" + pContainer + '/' + pDirectory

In [ ]:
def get_json_df(inputDF, json_column_name, spark_session):
    '''
    Description:
    This function provides the schema of json records and the dataframe to be used for flattening. If this doesnt happen, the source JSON String remains a string and cant be queries like JSON
        :param inputDF: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return df: dataframe to be used for flattening
    '''
    # creating a column transformedJSON to create an outer struct
    df1 = inputDF.withColumn('transformed_json', concat(lit("""{"transformed_json" :"""), inputDF[json_column_name], lit("""}""")))
    json_df = spark_session.read.json(df1.rdd.map(lambda row: row.transformed_json))
    # get schema
    json_schema = json_df.schema
    
    #Return a dataframe with the orignal column name but with proper JSON typed data
    df = df1.drop(json_column_name)\
        .withColumn(json_column_name, from_json(col('transformed_json'), json_schema))\
        .drop('transformed_json')\
        .select(f'{json_column_name}.*', '*')\
        .drop(json_column_name)\
        .withColumnRenamed("transformed_json", json_column_name)
    return df

In [ ]:
def execute_autoflatten(df, json_column_name):
    '''
    Description:
    This function executes the core autoflattening operation

    :param df: [type: pyspark.sql.dataframe.DataFrame] dataframe to be used for flattening
    :param json_column_name: [type: string] name of the column with json string

    :return df: DataFrame containing flattened records
    '''
    # gets all fields of StructType or ArrayType in the nested_fields dictionary
    nested_fields = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
    ])

    # repeat until all nested_fields i.e. belonging to StructType or ArrayType are covered
    while nested_fields:
        # if there are any elements in the nested_fields dictionary
        if nested_fields:
            # get a column
            column_name = list(nested_fields.keys())[0]
            # if field belongs to a StructType, all child fields inside it are accessed
            # and are aliased with complete path to every child field
            if isinstance(nested_fields[column_name], StructType):
                unnested = [col(column_name + '.' + child).alias(column_name + '>' + child) for child in [ n.name for n in  nested_fields[column_name]]]
                df = df.select("*", *unnested).drop(column_name)
            # else, if the field belongs to an ArrayType, an explode_outer is done
            elif isinstance(nested_fields[column_name], ArrayType):
                df = df.withColumn(column_name, explode_outer(column_name))

        # Now that df is updated, gets all fields of StructType and ArrayType in a fresh nested_fields dictionary
        nested_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
        ])

    # renaming all fields extracted with json> to retain complete path to the field
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace("transformedJSON", json_column_name))
    return df

In [ ]:
def execute_autoflatten_json(input_df, json_column_name, spark_session):
    '''
    Description:
    This function executes the flattening of json records in the dataframe. It calls the `get_json_df` and `execute_autoflatten` functions to do this. 
        :param input_df: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return unstd_df: contains flattened dataframe with unstandardized column name format
    '''
    json_df = get_json_df(input_df, json_column_name, spark_session)
    unstd_df = execute_autoflatten(json_df, json_column_name)
    return unstd_df

In [ ]:
class Log4jWrapper(object):
    """
    ------------------------------
    Description
    ------------------------------
    A class that creates a callable instance of the PySpark built in Log4j JVM logging object.
    Log4j is a Java-based logging utility part of the Apache Logging Services. 
    
    ------------------------------
    Parameters
    ------------------------------
    spark:
        SparkSession object.
    """

    def __init__(self, spark):
        """        
        ------------------------------
        Description
        ------------------------------
        Initialise the class with configuration varaibles and generate the custom log4j logger object.
        Will create the message prefix standard value to be used when logging errors, warnings or informational messages in this format.
            'Custom Message Logged from <applicationName: <ApplicationName> | applicationID: <ID> | notebookName: <NotebookName>>
        
        ------------------------------
        Parameters
        ------------------------------
        spark:
            The currently active SparkSession object        
            
        ------------------------------
        Return Value(s)
        ------------------------------
        None
        
        ------------------------------
        Example usage
        ------------------------------
        vLog4jWrapper = Log4jWrapper(spark)

        vLog4jWrapper.info('Test info logging')
        vLog4jWrapper.error('Test error logging')
        vLog4jWrapper.warning('Test warning logging')
        
        """
        # Get the currently active Spark Application details with which to prefix all messages. Store the details in variable(s) that other methods in this class can utilise
        self.sparkConfiguration = spark.sparkContext.getConf()
        self.applicationID = self.sparkConfiguration.get('spark.app.id')
        self.applicationName = self.sparkConfiguration.get('spark.app.name')
        
        # Get the notebook name where the class is being instantiated in. Store the details in variables that other methods in this class can utilise
        self.notebookName = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        #Create the custom message prefix to attach to all messages sent to the methods of this class. Store the details in variables that other methods in this class can utilise
        self.message_prefix = 'Custom Message Logged from <applicationName: ' + self.applicationName + ' | applicationID: ' + self.applicationID + ' | notebookName: ' + self.notebookName + '>: '
        
        #Get the reference to the Log4h object in this spark application
        log4j = spark._jvm.org.apache.log4j
        
        # Using the Log4j object, get an instance of the logger, passing in the custom message prefix defined above
        self.message_prefix = '<' + self.applicationName + ' ' + self.applicationID + ' ' + self.notebookName + '>'
        
        #Create the logger object instance used by all methods of this class
        self.logger = log4j.LogManager.getLogger(self.message_prefix)

    def error(self, message):
        """
        ------------------------------
        Description
        ------------------------------
        Log an error to the Log4j log using the logger defined int the init function.
        The error message should be clear as to what is most likely causing the problem. 
        Provide as much context to the current state of the application as possible to help guide troubleshooting. 
        
        ------------------------------
        Parameters
        ------------------------------
        message:
            The error message to log that describes the current state of the code, data, variables etc. 
            
        ------------------------------
        Return Value(s)
        ------------------------------
        None        
        """
        
        self.logger.error(message)
        
        return None

    def warning(self, message):
        """
        ------------------------------
        Description
        ------------------------------
        Log a Warning to the Log4j log using the logger defined int the init function.
        Warnings should state when best practices are not being followed which will not necesarily cause errors. 
        
        ------------------------------
        Parameters
        ------------------------------
        message:
            The warning message to log that describes the current state of the code, data, variables etc. 
            
        ------------------------------
        Return Value(s)
        ------------------------------
        None        
        """        

        self.logger.warn(message)
        
        return None

    def info(self, message):
        """
        ------------------------------
        Description
        ------------------------------
        Log an informational message to the Log4j log using the logger defined int the init function.
        Typically used for debugging/troubleshooting and general program state logging. 
        Use to log variable values or the current location in the notebook to the Log4J log. 
        
        ------------------------------
        Parameters
        ------------------------------
        message:
            The message to log that describes the current state of the code, data, variables etc. 
            
        ------------------------------
        Return Value(s)
        ------------------------------
        None        
        """            

        self.logger.info(message)
        
        return None